In [1]:
!pip install ratelimit

  Using cached ratelimit-2.2.1.tar.gz (5.3 kB)
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5891 sha256=5a9496461f7373cf3b958e75a388ced9a17405af1bf30792433836b82d368db9
  Stored in directory: /Users/IshanSharma/Library/Caches/pip/wheels/8e/34/5e/1dd3d652594bdf5df01109a683a455121d0b726978051bf720
Successfully built ratelimit


In [3]:
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata

# Importing libraries you need to install
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm
import threading
import time as t_time
from ratelimit import limits

### Creating a name & ticker df for 21 companies 

In [30]:
#read the ticker library of all the tikers into ticker_library
ticker_library = pd.read_csv('/Users/IshanSharma/Desktop/College Documents/Third Year/SEM V/CD Project/tickers.csv')

#read the sp500 components into ticker_selected, 'name' is the company name and ticker is company's ticker
ticker_selected = pd.read_csv('/Users/IshanSharma/Desktop/College Documents/Third Year/SEM V/CD Project/component_stocks.csv',header = None)
ticker_selected.columns = ['name','ticker']

#display a sample of ticker_selected
display(ticker_selected)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,ticker
0,Apple Inc.,AAPL
1,Abbott Laboratories,ABT
2,Adobe Inc.,ADBE
3,Amazon.com Inc.,AMZN
4,Boeing Company,BA
5,Bank of America Corp,BAC
6,Citigroup Inc.,C
7,Colgate-Palmolive Company,CL
8,Walt Disney Company,DIS
9,eBay Inc.,EBAY


### Fetching CIK numbers for the stocks

In [31]:
#build a ticker_cik_df dataframe to store ticker and its cik number
ticker_cik_df = pd.DataFrame()

#store all the tickers in a ticker_list
ticker_list = ticker_selected.ticker.unique()

#build a list cik_list for cik
cik_list = []

for ticker in ticker_list:    
    try:
        #for a given ticker, find its cik number through th ticker library
        cik_list.append(list(ticker_library[ticker_library.ticker == ticker].secfilings)[0][-10:])
        
    except:
        #if could not find cik, give it a empty cik
        cik_list.append('')

#write cik_list and ticker_list to the dataframe ticker_cik_df
ticker_cik_df['cik'] = cik_list
ticker_cik_df['ticker'] = ticker_list

#delete the tickers with empty cik number
ticker_cik_df = ticker_cik_df[ticker_cik_df['cik'] != '']

#display a sample of ticker_cik_df
ticker_cik_df

,cik,ticker
0,0000320193,AAPL
1,0000001800,ABT
2,0000796343,ADBE
3,0001018724,AMZN
4,0000012927,BA
5,0000070858,BAC
6,0000831001,C
7,0000021665,CL
8,0001744489,DIS
9,0001065088,EBAY


### Function to Scrape

In [32]:
def WriteLogFile(log_file_name, text):
    '''
    Helper function.
    Writes a log file with all notes and
    error messages from a scraping "session".
    
    Parameters
    ----------
    log_file_name : str
        Name of the log file (should be a .txt file).
    text : str
        Text to write to the log file.
        
    Returns
    -------
    None.
    
    '''
    with open(log_file_name, "a") as log_file:
        log_file.write(text)

    return

In [7]:
def Scrape10K(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    '''
    Scrapes all 10-Ks and 10-K405s for a particular 
    CIK from EDGAR.
    
    Parameters
    ----------
    browse_url_base : str
        Base URL for browsing EDGAR.
    filing_url_base : str
        Base URL for filings listings on EDGAR.
    doc_url_base : str
        Base URL for one filing's document tables
        page on EDGAR.
    cik : str
        Central Index Key.
    log_file_name : str
        Name of the log file (should be a .txt file).
        
    Returns
    -------
    None.
    
    '''
    
    # Check if we've already scraped this CIK
    try:
        os.mkdir(cik)
    except OSError:
        print("Already scraped CIK", cik)
        return
    
    # If we haven't, go into the directory for that CIK
    os.chdir(cik)
    
    #I avoid print here
    """
    print('Scraping CIK', cik)
    """
    hdr = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36'}
    # Request list of 10-K filings
    res = requests.get(browse_url_base % cik,headers=hdr)
    
    # If the request failed, log the failure and exit
    if res.status_code != 200:
        os.chdir('..')
        os.rmdir(cik) # remove empty dir
        text = "Request failed with error code " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        WriteLogFile(log_file_name, text)
        return

    # If the request doesn't fail, continue...
    
    # Parse the response HTML using BeautifulSoup
    soup = bs.BeautifulSoup(res.text, "lxml")

    # Extract all tables from the response
    html_tables = soup.find_all('table')
    
    # Check that the table we're looking for exists
    # If it doesn't, exit
    if len(html_tables)<3:
        os.chdir('..')
        return
    
    # Parse the Filings table
    filings_table = pd.read_html(str(html_tables[2]), header=0)[0]
    filings_table['Filings'] = [str(x) for x in filings_table['Filings']]

    # Get only 10-K and 10-K405 document filings
    filings_table = filings_table[(filings_table['Filings'] == '10-K') | (filings_table['Filings'] == '10-K405')]

    # If filings table doesn't have any
    # 10-Ks or 10-K405s, exit
    if len(filings_table)==0:
        os.chdir('..')
        return
    
    # Get accession number for each 10-K and 10-K405 filing
    filings_table['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in filings_table['Description']]

    # Iterate through each filing and 
    # scrape the corresponding document...
    for index, row in filings_table.iterrows():
        
        # Get the accession number for the filing
        acc_no = str(row['Acc_No'])
        
        # Navigate to the page for the filing
        docs_page = requests.get(filing_url_base % (cik, acc_no),headers=hdr)
        
        # If request fails, log the failure
        # and skip to the next filing
        if docs_page.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue

        # If request succeeds, keep going...
        
        # Parse the table of documents for the filing
        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        # Get the 10-K and 10-K405 entries for the filing
        docs_table = docs_table[(docs_table['Type'] == '10-K') | (docs_table['Type'] == '10-K405')]
        
        # If there aren't any 10-K or 10-K405 entries,
        # skip to the next filing
        if len(docs_table)==0:
            continue
        # If there are 10-K or 10-K405 entries,
        # grab the first document
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
        
        # If that first entry is unavailable,
        # log the failure and exit
        if str(docname) == 'nan':
            os.chdir('..')
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue       
        
        # If it is available, continue...
        
        # Request the file
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname),headers=hdr)
        
        # If the request fails, log the failure and exit
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
        
        # If it succeeds, keep going...
        
        # Save the file in appropriate format
        """
        date = str(row['Filing Date'])
        filename = cik + '_' + date + '.txt'
        html_file = open(filename, 'a')
        html_file.write(file.text)
        html_file.close()
        """
        if '.txt' in docname:
            # Save text as TXT
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            # Save text as HTML
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        
    # Move back to the main 10-K directory
    os.chdir('..')
        
    return

In [33]:
pathname_10k = '/Users/IshanSharma/Desktop/College Documents/Third Year/SEM V/CD Project/Scrape_Data_New/10_k/'

In [36]:
ticker_new=ticker_cik_df[:16]

In [34]:
# Run the function to scrape 10-Ks

# Define parameters
browse_url_base_10k = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-K'
filing_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

# Set correct directory
try:
    os.chdir(pathname_10k)
except Exception:
    os.makedirs(pathname_10k)
    os.chdir(pathname_10k)

# Initialize log file
# (log file name = the time we initiate scraping session)
time = strftime("%Y-%m-%d %Hh%Mm%Ss", gmtime())
log_file_name = 'log '+time+'.txt'
with open(log_file_name, 'a') as log_file:
    log_file.close()

# Iterate over CIKs and scrape 10-Ks
for cik in tqdm(ticker_cik_df['cik']):
    Scrape10K(browse_url_base=browse_url_base_10k, 
          filing_url_base=filing_url_base_10k, 
          doc_url_base=doc_url_base_10k, 
          cik=cik,
          log_file_name=log_file_name)
    

#return to the main menu
os.chdir('..')
os.chdir('..')

 81%|████████  | 17/21 [23:18<05:28, 82.24s/it]


KeyboardInterrupt: 

# convert html files to pure txt: preparing function

In [40]:
def RemoveNumericalTables(soup):
    
    '''
    Removes tables with >15% numerical characters.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup
        with numerical tables removed.
        
    '''
    
    # Determines percentage of numerical characters
    # in a table
    def GetDigitPercentage(tablestring):
        if len(tablestring)>0.0:
            numbers = sum([char.isdigit() for char in tablestring])
            length = len(tablestring)
            return numbers/length
        else:
            return 1
    
    # Evaluates numerical character % for each table
    # and removes the table if the percentage is > 15%
    [x.extract() for x in soup.find_all('table') if GetDigitPercentage(x.get_text())>0.15]
    
    return soup

In [41]:
def RemoveTags(soup):
    
    '''
    Drops HTML tags, newlines and unicode text from
    filing text.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    text : str
        Filing text.
        
    '''
    
    # Remove HTML tags with get_text
    text = soup.get_text()
    
    # Remove newline characters
    text = text.replace('\n', ' ')
    
    '''
    Adding a replace to convert all excecute space to one space 
    '''
    text = re.sub(r'\s', ' ', text)
    
    # Replace unicode characters with their
    # "normal" representations
    text = unicodedata.normalize('NFKD', text)
    
    return text

In [42]:
def ConvertHTML(cik):
    
    '''
    Removes numerical tables, HTML tags,
    newlines, unicode text, and XBRL tables.
    
    Parameters
    ----------
    cik : str
        Central Index Key used to scrape files.
    
    Returns
    -------
    None.
    
    '''
    
    # Look for files scraped for that CIK
    try: 
        os.chdir(cik)
    # ...if we didn't scrape any files for that CIK, exit
    except FileNotFoundError:
        print("Could not find directory for CIK", cik)
        return
    """ avoid print"""    
    #print("Parsing CIK %s..." % cik)
    
    parsed = False # flag to tell if we've parsed anything
    
    # Try to make a new directory within the CIK directory
    # to store the text representations of the filings
    try:
        os.mkdir('rawtext')
    # If it already exists, continue
    # We can't exit at this point because we might be
    # partially through parsing text files, so we need to continue
    except OSError:
        pass
    
    # Get list of scraped files
    # excluding hidden files and directories
    file_list = [fname for fname in os.listdir() if not (fname.startswith('.') | os.path.isdir(fname))]
    
    # Iterate over scraped files and clean
    for filename in file_list:
            
        # Check if file has already been cleaned
        new_filename = filename.replace('.html', '.txt')
        text_file_list = os.listdir('rawtext')
        if new_filename in text_file_list:
            continue
        
        # If it hasn't been cleaned already, keep going...
        
        # Clean file
        with open(filename, 'r',encoding='utf-8') as file:
            parsed = True
            soup = bs.BeautifulSoup(file.read(), "lxml")
            soup = RemoveNumericalTables(soup)
            
            #add my delete reference function here
            #soup = delete_reference(soup)
            
            text = RemoveTags(soup)
            with open('rawtext/'+new_filename, 'w',encoding='utf-8') as newfile:
                newfile.write(text)
    
    # If all files in the CIK directory have been parsed
    # then log that
    if parsed==False:
        print("Already parsed CIK", cik)
    
    os.chdir('..')
    return

In [43]:
'''
In the documents, there are some parts not in the Management Discussion but contain reference to this part,like:
'For a discussion of our contractual obligations, see “Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations,”'' 

That is why I need a function to delete all the reference parts of MD&A.

'''

#compose a function to delete the references
def delete_reference(soup):
    
    #define string reference to the management's discussion parts
    management_str = "Financial Condition and Results of Operations"
                    
    #define string reference to the financial statement parts, which is what we need for parsing
    financial_str = "and Supplementary Data"
    
    #given the none reference length for comparision
    length_1 = len('Management’s Discussion and Analysis of Financial Condition and Results of Operations')
    
    #given the none reference length for comparision
    length_2 = len('Financial Statements and Supplementary Data')
    
    #find all the parts contains this reference
    for part in soup.find_all(text = re.compile(management_str)):
        
        #if the lenth is much greater than the original string length, then it's a reference, and we should delete it
        if len(part) > length_1:
            
            #we should extract these text
            part.extract()
            
    #the same should be operated to the financial statements
    for part in soup.find_all(text = re.compile(financial_str)):
        if len(part) > length_2:
            part.extract()
    
    #return soup for futher parsing
    return soup


In [44]:
# For 10-Ks...
# -*- coding: utf-8 -*-
os.chdir(pathname_10k)

# Iterate over CIKs and clean HTML filings
for cik in tqdm(ticker_new['cik']):
    ConvertHTML(cik)
os.chdir('..')
os.chdir('..')

 50%|█████     | 8/16 [05:05<06:19, 47.46s/it]

Already parsed CIK 0001744489


100%|██████████| 16/16 [09:28<00:00, 35.50s/it]


## EXTRA


In [37]:
def try_section(data):
    management_str_list = list(combs['FIRST ITEM'].dropna())
    financial_str_list = list(combs['SECOND ITEM'].dropna())
    for i in range(len(management_str_list)):
        management_str_list[i] = management_str_list[i].upper()
        management_str = management_str_list[i]
        management_str = re.sub('\s+',' ',management_str)
        last_management = data.rfind(management_str)
        if (last_management == -1):
            continue
        else:
            break
    for i in range(len(financial_str_list)):
        financial_str_list[i] = financial_str_list[i].upper()
        financial_str = financial_str_list[i]
        financial_str = re.sub('\s+',' ',financial_str)
        if data.count(financial_str) == 0:
            financial_str = "FINANCIAL STATEMENTS AND SUPPLEMENTAL DATA"
        last_financial = data.rfind(financial_str)
        if (last_financial == -1):
            continue
        else:
            break
    return last_management, last_financial

In [38]:
#write a function to parse the raw text to graB item7 and item 7a informations:

#set input as the path of the txt file
def grab_md_10k(text_path):
    
    #open the text file and read file as upper string
    with open(text_path, encoding = "utf8") as f:
        data = f.read().upper()

    #normalize all the blanks
    data = re.sub('\s+', ' ', data)
    management_str = ["ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 
                      "Discussion and Analysis of Financial Condition and Results of Operations".upper(),
                      "Analysis of Financial Condition and Results of Operations".upper(),
                      "Discussion and Analysis of Financial Condition".upper()]
    financial_str = ["ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA", 
                     "FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA", 
                     "FINANCIAL STATEMENTS AND SUPPLEMENTAL DATA"]
    for j in financial_str:
        last_financial = data.rfind(j)
        if (last_financial == -1):
            continue
        else:
            break
    for i in management_str:
        last_management = data.rfind(i)
        if (last_management == -1):
            continue
        else:
            break
    
    
    '''
    after deleting references, there should be 1 or 2 titles just above the information we need. One is in the catalog, the other 
    is in the context.
    '''
    
    #if last management string happens after the financial statement, then we cannot grab information
    if last_management > last_financial:
        
        #then we should return none 
        return None

    #else we can get a dialogue of management discussion
    else:
        return data[last_management:last_financial]    


In [39]:
#iterate through the ciks to grab 10K information
for cik in ticker_new['cik']: 
            
    
    #file name of 
    file_list = os.listdir(pathname_10k + cik)
    
    #iterate through file name to grab managemnent discussions
    for file_name in file_list:
                
        #complete a path for the text file
        text_path = pathname_10k + cik + '/' + file_name
        #grab the management discussion part and convert it to a string
        data = grab_md_10k(text_path)
        
        #decide whether we have data to write into file
        if data != None:
        
            #move to the directory for grabbed_text 
            try:
                os.chdir(pathname_10k + cik + '/grabbed_text/')
            
            #if there isn't a directory, we compose one    
            except Exception:
                os.makedirs(pathname_10k + cik + '/grabbed_text/')
                os.chdir(pathname_10k + cik + '/grabbed_text/')
        
            #open a txt file to write data into
            file = open(file_name, 'a', encoding = "utf-8")
            
            #write data into file    
            file.write(data)
        
            #close file and return to the main directory
            file.close()
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')

In [ ]:
#iterate through the ciks to grab 10K information
for cik in ticker_cik_sample:
    
    
    #file name of 
    file_list = os.listdir(pathname_10k + cik + '/rawtext/')
    #iterate through file name to grab managemnent discussions
    for file_name in file_list:
        
        #complete a path for the text file
        text_path = pathname_10k + cik + '/rawtext/' + file_name
        #grab the management discussion part and convert it to a string
        data = grab_md_10k(text_path)
        
        #decide whether we have data to write into file
        if data != None:
        
            #move to the directory for grabbed_text 
            try:
                os.chdir(pathname_10k + cik + '/grabbed_text/')
            
            #if there isn't a directory, we compose one    
            except Exception:
                os.makedirs(pathname_10k + cik + '/grabbed_text/')
                os.chdir(pathname_10k + cik + '/grabbed_text/')
        
            #open a txt file to write data into
            file = open(file_name, 'a', encoding = "utf-8")
            
            #write data into file    
            file.write(data)
        
            #close file and return to the main directory
            file.close()
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')